In [132]:
import pandas as pd
import datetime

In [133]:
workout_df = pd.read_csv("../1. Inspect, Clean and Validate Dataset/CleanedData/Cleaned_StrongApp.csv")
workout_df["Date"] = pd.to_datetime(workout_df["Date"], errors="coerce")
workout_df["Date"] = workout_df["Date"].dt.normalize()

Drop some unneeded columns

In [134]:
workout_df = workout_df[["Date", "Exercise Name", "Weight", "Reps"]]
workout_df

,Date,Exercise Name,Weight,Reps
0,2024-09-19,Hip Abductor (Machine),55.000000,10.0
1,2024-09-19,Hip Abductor (Machine),65.000000,10.0
2,2024-09-19,Hip Abductor (Machine),65.000000,10.0
3,2024-09-19,Seated Leg Curl (Machine),77.160000,8.0
4,2024-09-19,Seated Leg Curl (Machine),77.160000,11.0
...,...,...,...,...
1485,2025-08-21,Leg Extension (Machine),22.679619,15.0
1486,2025-08-21,Leg Extension (Machine),18.143695,18.0
1487,2025-08-21,Leg Extension (Machine),22.679619,9.0
1488,2025-08-21,Leg Extension (Machine),22.679619,13.0


I want to visualise my estimated 1 rep max progression throughout the period that I have been working out.
Because I've done slightly different exercises, I need to generalise them
- For example, "Hammer Curl (Dumbbell)" and "Bicep Curl (Dumbbell)" both primarily work the biceps so I can group these together.
It is important to also differentiate between exercises that used machines and exercises that used free weights
- Free weights require additional muscles to support and balance throughout the movement
- For example, "Chest Press (Machine)" and "Bench Press (Dumbbell)" primarily work the pecs but can't be grouped together

I need to manually go through and adjust all of this data

In [135]:
def primary_muscle_used(exercise: str) -> str:
	muscle_used = ""
	match exercise:
		case "Hip Abductor (Machine)":
			muscle_used = "abductors"
		case "Calf Stretch":
			muscle_used = "n/a"
		case "Seated Leg Curl (Machine)":
			muscle_used = "hamstrings"
		case "Lat Pulldown (Cable)":
			muscle_used = "lats"
		case "Triceps Press down (Rope)":
			muscle_used = "triceps"
		case "Lateral Raise (Dumbbell)":
			muscle_used = "delts"
		case "Hanging Knee Raise":
			muscle_used = "abs"
		case "Chest Press (Machine)":
			muscle_used = "pecs"
		case "Hammer Curl (Dumbbell)":
			muscle_used = "biceps"
		case "Abdominal Machine":
			muscle_used = "abs"
		case "Triceps Extension (Dumbbell)":
			muscle_used = "triceps"
		case "Leg Extension (Machine)":
			muscle_used = "quads"
		case "Hip Adductor (Machine)":
			muscle_used = "adductors"
		case "Triceps Extension (Cable)":
			muscle_used = "triceps"
		case "Lateral Raise (Cable)":
			muscle_used = "delts"
		case "Chest Fly":
			muscle_used = "pecs"
		case "Preacher Curl (Dumbbell)":
			muscle_used = "biceps"
		case "Chest Fly (Dumbbell)":
			muscle_used = "pecs"
		case "Hip Thrust (Barbell)":
			muscle_used = "hip thrust"
		case "Bench Press (Barbell)":
			muscle_used = "pecs"
		case "Preacher Curl (Barbell)":
			muscle_used = "biceps"
		case "Triceps Pushdown (Cable - Straight Bar)":
			muscle_used = "triceps"
		case "Triceps Dip (Assisted)":
			muscle_used = "triceps"
		case "Incline Curl (Dumbbell)":
			muscle_used = "biceps"
		case "Seated Calf Raise (Machine)":
			muscle_used = "calf"
		case "Incline Bench Press (Barbell)":
			muscle_used = "pecs"
		case "Back Squat (Smith Machine)":
			muscle_used = "Back Squat (Smith Machine)" # I know this isn't one muscle, but as this is a compound movement I am happy to separate it. I track this 1RM with my physio
		case "Front Squat (Smith Machine)":
			muscle_used = "Front Squat (Smith Machine)" # similar to above
		case "Lunge (Right Foot Forward, Smith Machine)":
			muscle_used = "Lunge (Right Foot Forward, Smith Machine)"
		case "Lunge (Left Foot Forwards, Smith Machine)": # similar to above
			muscle_used = "Lunge (Left Foot Forwards, Smith Machine)"
		case "Deadlift (Smith Machine)": # similar to above
			muscle_used = "Deadlift (Smith Machine)"
		case "Bent Over Row (Barbell)":
			muscle_used = "lats"
		case "Seated Leg Press (Machine)":
			muscle_used = "glutes/quads" # for this we need to look at another column. this exercise either biased the quads or the glutes
		case "Incline Bench Press (Smith Machine)":
			muscle_used = "pecs"
		case "Bench Press (Smith Machine)":
			muscle_used = "pecs"
		case "Bicep Curl (Dumbbell)":
			muscle_used = "biceps"
		case "Triceps Pushdown (^bar)":
			muscle_used = "triceps"
		case "Bench Press (Dumbbell)":
			muscle_used = "pecs"
		case "Incline Bench Press (Machine)":
			muscle_used = "pecs"
		case "Hack Squat":
			muscle_used = "Hack Squat" # again, a compound movement which targets multiple muscle groups
		case "Seated Calf Raise (Plate Loaded)":
			muscle_used = "calf"
		case "Cable Crunch":
			muscle_used = "abs"
		case _:
			muscle_used = "other"

	return muscle_used

In [136]:
workout_df["Primary Muscle Used"] = workout_df["Exercise Name"].apply(primary_muscle_used)

Although not noted in the data, workouts between 2025-06-02 and 2025-08-09 on seated leg press targetted the glutes

In [137]:
mask = (
    (workout_df["Exercise Name"] == "Seated Leg Press (Machine)") &
    (workout_df["Date"].between(datetime.datetime(2025, 6, 1), datetime.datetime(2025, 8, 9)))
)
workout_df.loc[mask, "Primary Muscle Used"] = "glutes"

Drop any rows which contain "Rest Timer". This just represents the rest time between sets and is not relevant

In [138]:
# workout_df = workout_df[workout_df["Set Order"] != "Rest Timer"]
# workout_df.reset_index(inplace=True, drop=True)
# workout_df = workout_df.drop("Set Order", axis=1)

We need to calculate the estimated one rep max

In [139]:
def estimated_one_rep_max(row: pd.Series):
	# using the Epley formula
	# 1RM = W•(1 + r/30)
	return round(row["Weight"] * (1+(row["Reps"]/30)))

In [140]:
workout_df["Epley 1RM"] = workout_df.apply(estimated_one_rep_max, axis=1)

Lets seperate the exercises depending on: machine or single arm, both arms
- for example "Lat Pulldown (Cable)" is a machine exercise
- "Chest Press (Machine)" is a machine exercise that uses both arms (so the weight used is representative of the strength of both arms)
- "Hammer Curl (Dumbbell)" is a free weight exercise, that uses single arm (so the weight is representative of the strength of one arm)

In [141]:
def machine_or_freeweight(exercise: str) -> int:
	"""
	:param exercise: str representing the name of the exercise
	:return: 1 if machine, 0 if free weight
	"""
	match exercise:
		case "Hip Abductor (Machine)":
			return 1
		case "Calf Stretch":
			return 0
		case "Seated Leg Curl (Machine)":
			return 1
		case "Lat Pulldown (Cable)":
			return 1
		case "Triceps Press down (Rope)":
			return 1
		case "Lateral Raise (Dumbbell)":
			return 0
		case "Hanging Knee Raise":
			return 0
		case "Chest Press (Machine)":
			return 1
		case "Hammer Curl (Dumbbell)":
			return 0
		case "Abdominal Machine":
			return 1
		case "Triceps Extension (Dumbbell)":
			return 0
		case "Leg Extension (Machine)":
			return 1
		case "Hip Adductor (Machine)":
			return 1
		case "Triceps Extension (Cable)":
			return 1
		case "Lateral Raise (Cable)":
			return 1
		case "Chest Fly":
			return 1
		case "Preacher Curl (Dumbbell)":
			return 0
		case "Chest Fly (Dumbbell)":
			return 0
		case "Hip Thrust (Barbell)":
			return 0
		case "Bench Press (Barbell)":
			return 0
		case "Preacher Curl (Barbell)":
			return 0
		case "Triceps Pushdown (Cable - Straight Bar)":
			return 1
		case "Triceps Dip (Assisted)":
			return 1
		case "Incline Curl (Dumbbell)":
			return 0
		case "Seated Calf Raise (Machine)":
			return 0
		case "Incline Bench Press (Barbell)":
			return 0
		case "Back Squat (Smith Machine)":
			return 1
		case "Front Squat (Smith Machine)":
			return 1
		case "Lunge (Right Foot Forward, Smith Machine)":
			return 1
		case "Lunge (Left Foot Forwards, Smith Machine)":  # similar to above
			return 1
		case "Deadlift (Smith Machine)":  # similar to above
			return 1
		case "Bent Over Row (Barbell)":
			return 0
		case "Seated Leg Press (Machine)":
			return 1
		case "Incline Bench Press (Smith Machine)":
			return 1
		case "Bench Press (Smith Machine)":
			return 1
		case "Bicep Curl (Dumbbell)":
			return 0
		case "Triceps Pushdown (^bar)":
			return 1
		case "Bench Press (Dumbbell)":
			return 0
		case "Incline Bench Press (Machine)":
			return 1
		case "Hack Squat":
			return 1
		case "Seated Calf Raise (Plate Loaded)":
			return 1
		case "Cable Crunch":
			return 1
		case _:
			return 1

In [142]:
workout_df["Machine Used"] = workout_df["Exercise Name"].apply(machine_or_freeweight)
workout_df["Machine Used"] = workout_df["Machine Used"].astype("int32")
workout_df

,Date,Exercise Name,Weight,Reps,Primary Muscle Used,Epley 1RM,Machine Used
0,2024-09-19,Hip Abductor (Machine),55.000000,10.0,abductors,73,1
1,2024-09-19,Hip Abductor (Machine),65.000000,10.0,abductors,87,1
2,2024-09-19,Hip Abductor (Machine),65.000000,10.0,abductors,87,1
3,2024-09-19,Seated Leg Curl (Machine),77.160000,8.0,hamstrings,98,1
4,2024-09-19,Seated Leg Curl (Machine),77.160000,11.0,hamstrings,105,1
...,...,...,...,...,...,...,...
1485,2025-08-21,Leg Extension (Machine),22.679619,15.0,quads,34,1
1486,2025-08-21,Leg Extension (Machine),18.143695,18.0,quads,29,1
1487,2025-08-21,Leg Extension (Machine),22.679619,9.0,quads,29,1
1488,2025-08-21,Leg Extension (Machine),22.679619,13.0,quads,33,1


Categorise the exercises

In [143]:
def full_weight(exercise: str) -> str:
	"""I'm not sure what way to better describe this but "does the weight represent half of the weight during the exercise (like when using dumbbells) or the full weight during the exercise (like a machine)"

	:param exercise: str representing the name of the exercise
	:return: category of lift
	"""
	match exercise:
		case "Hip Abductor (Machine)":
			return "Hip Abductor (Machine, All Variants)"
		case "Calf Stretch":
			return "Calf Stretch (Machine, All Variants)"
		case "Seated Leg Curl (Machine)":
			return "Seated Leg Curl (Machine, All Variants)"
		case "Lat Pulldown (Cable)":
			return "Lat Pulldown (All Variants)"
		case "Triceps Press down (Rope)":
			return "Triceps Pushdown (Cable, All Variants)"
		case "Lateral Raise (Dumbbell)":
			return "Lateral Raise (Dumbbell, All Variants)"
		case "Hanging Knee Raise":
			return "Hanging Knee Raise (Body Weight, All Variants)"
		case "Chest Press (Machine)":
			return "Chest Press (Machine, All Variants)"
		case "Hammer Curl (Dumbbell)":
			return "Biceps Curl (Dumbbell, All Variants)"
		case "Abdominal Machine":
			return "Abdominal (Machine, All Variants)"
		case "Triceps Extension (Dumbbell)":
			return "Triceps Extension (Dumbbell, All Variants)"
		case "Leg Extension (Machine)":
			return "Leg Extension (Machine, All Variants)"
		case "Hip Adductor (Machine)":
			return "Hip Adductor (Machine, All Variants)"
		case "Triceps Extension (Cable)":
			return "Triceps Extension (Cable, All Variants)"
		case "Lateral Raise (Cable)":
			return "Lateral Raise (Cable, All Variants)"
		case "Chest Fly":
			return "Chest Fly (Machine, All Variants)"
		case "Preacher Curl (Dumbbell)":
			return "Preacher Curl (Dumbbell, All Variants)"
		case "Chest Fly (Dumbbell)":
			return "Chest Fly (Dumbbell, All Variants)"
		case "Hip Thrust (Barbell)":
			return "Hip Thrust (Barbell, All Variants)"
		case "Bench Press (Barbell)":
			return "Bench Press (Barbell, All Variants)"
		case "Preacher Curl (Barbell)":
			return "Preacher Curl (Barbell, All Variants)"
		case "Triceps Pushdown (Cable - Straight Bar)":
			return "Triceps Pushdown (Cable, All Variants)"
		case "Triceps Dip (Assisted)":
			return "Triceps Dip (Machine, Assisted)"
		case "Incline Curl (Dumbbell)":
			return "Biceps Curl (Dumbbell, All Variants)"
		case "Seated Calf Raise (Machine)":
			return "Seated Calf Raise (Plate Loaded, All Variants)"
		case "Incline Bench Press (Barbell)":
			return "Incline Bench Press (Barbell, All Variants)"
		case "Back Squat (Smith Machine)":
			return "Back Squat (Smith Machine)"
		case "Front Squat (Smith Machine)":
			return "Front Squat (Smith Machine)"
		case "Lunge (Right Foot Forward, Smith Machine)":
			return "Lunge (Right Foot Forward, Smith Machine)"
		case "Lunge (Left Foot Forwards, Smith Machine)":  # similar to above
			return "Lunge (Left Foot Forwards, Smith Machine)"
		case "Deadlift (Smith Machine)":  # similar to above
			return "Deadlift (Smith Machine)"
		case "Bent Over Row (Barbell)":
			return "Bent Over Row (Barbell, All Variants)"
		case "Seated Leg Press (Machine)":
			return "Seated Leg Press (Machine, All Variants)"
		case "Incline Bench Press (Smith Machine)":
			return "Incline Bench Press (Machine, All Variants)"
		case "Bench Press (Smith Machine)":
			return "Bench Press (Machine, All Variants)"
		case "Bicep Curl (Dumbbell)":
			return "Biceps Curl (Dumbbell, All Variants)"
		case "Triceps Pushdown (^bar)":
			return "Triceps Pushdown (Cable, All Variants)"
		case "Bench Press (Dumbbell)":
			return "Bench Press (Dumbbell, All Variants)"
		case "Incline Bench Press (Machine)":
			return "Incline Bench Press (Machine)"
		case "Hack Squat":
			return "Hack Squat (Plate Loaded, All Variants)"
		case "Seated Calf Raise (Plate Loaded)":
			return "Seated Calf Raise (Plate Loaded, All Variants)"
		case "Cable Crunch":
			return "Cable Crunch (Cable, All Variants)"
		case _:
			return "n/a"

In [144]:
workout_df["Category"] = workout_df["Exercise Name"].apply(full_weight)
workout_df["Category"].value_counts()

Category
Lateral Raise (Dumbbell, All Variants)            182
Lat Pulldown (All Variants)                       180
Biceps Curl (Dumbbell, All Variants)              130
Triceps Pushdown (Cable, All Variants)            120
Chest Press (Machine, All Variants)                98
Seated Leg Press (Machine, All Variants)           66
Lateral Raise (Cable, All Variants)                66
Hip Abductor (Machine, All Variants)               65
Hip Adductor (Machine, All Variants)               63
Chest Fly (Machine, All Variants)                  62
Leg Extension (Machine, All Variants)              54
Triceps Extension (Cable, All Variants)            54
Seated Leg Curl (Machine, All Variants)            53
Preacher Curl (Dumbbell, All Variants)             40
Triceps Extension (Dumbbell, All Variants)         35
Bench Press (Dumbbell, All Variants)               26
Abdominal (Machine, All Variants)                  26
Hip Thrust (Barbell, All Variants)                 26
Bench Press (Barbel

The data is fully prepared for visualisation

In [145]:
workout_df.to_csv("./Data/StrongApp_WorkoutData.csv")